# Modelagem Estatística em indicadores socioeconomicos

**Objetivo:** estudar quais são os indicadores socioeconômicos que impactam a expectativa de vida das pessoas.

Os dados foram extraídos do link abaixo:

https://www.who.int/data/gho/data/themes/mortality-and-global-health-estimates/ghe-life-expectancy-and-healthy-life-expectancy


In [21]:
# Imports
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.preprocessing import StandardScaler 
import warnings
warnings.filterwarnings("ignore")

In [44]:
dados = pd.read_csv("/home/priscila/Downloads/1-Mat/dados/dataset_mat_p6.csv")
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2938 entries, 0 to 2937
Data columns (total 22 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Country                          2938 non-null   object 
 1   Year                             2938 non-null   int64  
 2   Status                           2938 non-null   object 
 3   Life expectancy                  2928 non-null   float64
 4   Adult Mortality                  2928 non-null   float64
 5   infant deaths                    2938 non-null   int64  
 6   Alcohol                          2744 non-null   float64
 7   percentage expenditure           2938 non-null   float64
 8   Hepatitis B                      2385 non-null   float64
 9   Measles                          2938 non-null   int64  
 10   BMI                             2904 non-null   float64
 11  under-five deaths                2938 non-null   int64  
 12  Polio               

In [33]:
dados.shape

(2938, 22)

In [45]:
dados.columns

Index(['Country', 'Year', 'Status', 'Life expectancy ', 'Adult Mortality',
       'infant deaths', 'Alcohol', 'percentage expenditure', 'Hepatitis B',
       'Measles ', ' BMI ', 'under-five deaths ', 'Polio', 'Total expenditure',
       'Diphtheria ', ' HIV/AIDS', 'GDP', ' Population',
       ' thinness  1-19 years', ' thinness 5-9 years',
       'Income composition of resources', 'Schooling'],
      dtype='object')

In [46]:
## Ajuste inicial dos dados
# Renomeando colunas
dados = dados.rename(columns={'Country': 'pais', 
                              'Year': 'ano',
                              'Status': 'status',
                              'Life expectancy ': 'expectativa_vida',
                              'Adult Mortality': 'mortalidade_adulta',
                              'infant deaths': 'mortalidade_infantil',
                              'Alcohol': 'alcool',
                              'percentage expenditure': 'percentual_gasto',
                              'Hepatitis B': 'hepatite_b',
                              'Measles ': 'sarampo',
                              ' BMI ': 'imc',
                              'under-five deaths ': 'mortalidade_menores_5_anos',
                              'Polio': 'polio',
                              'Total expenditure': 'gasto_total',
                              'Diphtheria ': 'difteria',
                              ' HIV/AIDS': 'hiv_aids',
                              'GDP': 'pib',
                              ' Population': 'populacao',
                              ' thinness  1-19 years': 'magreza_19',
                              ' thinness 5-9 years': 'magreza_9',
                              'Income composition of resources': 'composicao_renda',
                              'Schooling': 'escolaridade'})

In [47]:
dados.columns

Index(['pais', 'ano', 'status', 'expectativa_vida', 'mortalidade_adulta',
       'mortalidade_infantil', 'alcool', 'percentual_gasto', 'hepatite_b',
       'sarampo', 'imc', 'mortalidade_menores_5_anos', 'polio', 'gasto_total',
       'difteria', 'hiv_aids', 'pib', 'populacao', 'magreza_19', 'magreza_9',
       'composicao_renda', 'escolaridade'],
      dtype='object')

In [52]:
dados_dict = {
    "pais": "País de origem dos dados.",
    "expectativa_vida": "Expectativa de vida ao nascer, em anos.",
    "ano": "Ano em que os dados foram coletados.",
    "status": "Status de desenvolvimento do país ('Developing' para países em desenvolvimento, 'Developed' para países desenvolvidos).",
    "mortalidade_adulta": "Taxa de mortalidade de adultos entre 15 e 60 anos por 1000 habitantes.",
    "mortalidade_infantil": "Número de mortes de crianças com menos de 5 anos por 1000 nascidos vivos.",
    "alcool": "Consumo de álcool per capita (litros de álcool puro por ano).",
    "hepatite_b": "Cobertura de vacinação contra hepatite B em crianças de 1 ano (%).",
    "sarampo": "Número de casos de sarampo relatados por 1000 habitantes.",
    "imc": "Índice médio de massa corporal da população adulta.",
    "polio": "Cobertura de vacinação contra poliomielite em crianças de 1 ano (%).",
    "difteria": "Cobertura de vacinação contra difteria, tétano e coqueluche (DTP3) em crianças de 1 ano (%).",
    "hiv_aids": "Prevalência de HIV na população adulta (%).",
    "pib": "Produto Interno Bruto per capita (em dólares americanos).",
    "gasto_total": "Gasto total em saúde como porcentagem do PIB.",
    "magreza_19": "Prevalência de magreza em crianças e adolescentes de 10 a 19 anos (%).",
    "magreza_9": "Prevalência de magreza em crianças de 5 a 9 anos (%).",
    "escolaridade": "Número médio de anos de escolaridade.",
    "populacao": "População total do país."
}
len(dados_dict)

19

In [53]:
dados.sample(5)

,pais,ano,status,expectativa_vida,mortalidade_adulta,mortalidade_infantil,alcool,percentual_gasto,hepatite_b,sarampo,...,polio,gasto_total,difteria,hiv_aids,pib,populacao,magreza_19,magreza_9,composicao_renda,escolaridade
2560,Tajikistan,2008,Developing,66.4,178.0,10,0.32,4.153363,86.0,0,...,87.0,5.58,86.0,0.3,76.914128,739728.0,3.8,3.8,0.592,10.8
1589,Malaysia,2012,Developing,74.5,129.0,3,0.53,101.609245,97.0,1868,...,97.0,4.10,97.0,0.1,1779.496410,2917456.0,7.9,7.7,0.776,12.9
2337,Slovakia,2007,Developed,74.4,14.0,0,10.58,0.000000,99.0,0,...,99.0,7.76,99.0,0.1,NaN,NaN,1.3,1.4,0.802,14.5
270,Belize,2001,Developing,68.2,21.0,0,4.90,251.658693,96.0,0,...,96.0,4.50,96.0,0.4,3419.275719,254984.0,3.7,3.7,0.677,11.8
2913,Zambia,2008,Developing,55.7,45.0,31,2.12,153.678375,87.0,140,...,9.0,4.87,87.0,11.9,1369.682490,1382517.0,6.8,6.7,0.504,11.4


## Análise Exploratória

In [54]:
dados.describe()

,ano,expectativa_vida,mortalidade_adulta,mortalidade_infantil,alcool,percentual_gasto,hepatite_b,sarampo,imc,mortalidade_menores_5_anos,polio,gasto_total,difteria,hiv_aids,pib,populacao,magreza_19,magreza_9,composicao_renda,escolaridade
count,2938.000000,2928.000000,2928.000000,2938.000000,2744.000000,2938.000000,2385.000000,2938.000000,2904.000000,2938.000000,2919.000000,2712.00000,2919.000000,2938.000000,2490.000000,2.286000e+03,2904.000000,2904.000000,2771.000000,2775.000000
mean,2007.518720,69.224932,164.796448,30.303948,4.602861,738.251295,80.940461,2419.592240,38.321247,42.035739,82.550188,5.93819,82.324084,1.742103,7483.158469,1.275338e+07,4.839704,4.870317,0.627551,11.992793
std,4.613841,9.523867,124.292079,117.926501,4.052413,1987.914858,25.070016,11467.272489,20.044034,160.445548,23.428046,2.49832,23.716912,5.077785,14270.169342,6.101210e+07,4.420195,4.508882,0.210904,3.358920
min,2000.000000,36.300000,1.000000,0.000000,0.010000,0.000000,1.000000,0.000000,1.000000,0.000000,3.000000,0.37000,2.000000,0.100000,1.681350,3.400000e+01,0.100000,0.100000,0.000000,0.000000
25%,2004.000000,63.100000,74.000000,0.000000,0.877500,4.685343,77.000000,0.000000,19.300000,0.000000,78.000000,4.26000,78.000000,0.100000,463.935626,1.957932e+05,1.600000,1.500000,0.493000,10.100000
50%,2008.000000,72.100000,144.000000,3.000000,3.755000,64.912906,92.000000,17.000000,43.500000,4.000000,93.000000,5.75500,93.000000,0.100000,1766.947595,1.386542e+06,3.300000,3.300000,0.677000,12.300000
75%,2012.000000,75.700000,228.000000,22.000000,7.702500,441.534144,97.000000,360.250000,56.200000,28.000000,97.000000,7.49250,97.000000,0.800000,5910.806335,7.420359e+06,7.200000,7.200000,0.779000,14.300000
max,2015.000000,89.000000,723.000000,1800.000000,17.870000,19479.911610,99.000000,212183.000000,87.300000,2500.000000,99.000000,17.60000,99.000000,50.600000,119172.741800,1.293859e+09,27.700000,28.600000,0.948000,20.700000


### Análise da variável alvo

In [ ]:
dados['expectativa_vida'].hist(bins=20)
plt.title('Distribuição da Expectativa de Vida')
plt.xlabel('Expectativa de Vida (anos)')
plt.ylabel('Frequência')
plt.show()